# Partitioning an array
### Sriram Sankaranarayanan <srirams@colorado.edu>

In this notebook, we carefully go over the existing partition algorithms to be used in quicksort to understand some of the finer details.

In particular, we covered two algorithms in class:

- Lomuto partitioning scheme: this approach partitions from left to right and maintains two regions of array elements.

- Hoare partitioning scheme: this approach partitions starting from both ends of the array that meet in the middle.

This notebook will explain the loop invariants that will be maintained as we partition.

## What is Partitioning?

Consider an array $a$:

$$ a: [ 2, 1, 3, 5, 6, 7, 8, 2, 3, 2, \mathbf{4} ] $$

We choose a special element called the _pivot_. Here the pivot will be chosen as the last element $4$.
In general, let the array have $n$ elements and be written as

$$ a: \left[\ a[0], a[1], \ldots, \mathbf{a[n-1]}\ \right]\,. $$

Let $x = a[n-1]$ represent the chosen pivot.

The act of partitioning _permutes_ or rearranges the array $a$ in-place so that after partitioning, the new array $\hat{a}$ obtained after partitioning has the following structure (for convenience, we pretend this is a new array and call it $\hat{a}$, but our implementation will not create new space).

$$\hat{a}:\ \left[\ \hat{a}[0], \ldots, \mathbf{\hat{a}[j]}, \ldots, \hat{a}[n-1]\ \right]$$

- $(\forall\ k \in [0,j)\ a[k] \leq x$ (_every element in the range from $0$ to $j-1$ is less than or equal to $x$_ )
- $(\forall\ l \in (j,n-1] )\ a[l] \geq x$, (_every element in the range from $j+1$ to $n-1$ is greater than or equal to $x$_)
- $a[j] == x$ (_ the element at $j$ equals $x$_ ).


Let us write a function called 'isPartitioned' that given $a$, $j$ checks that $a$ has been indeed partitioned at location $j$. We will never need to call such a function but it is good to test whether our algorithms are correct.


In [1]:
def isPartitioned(a, j, verbose=True):
    # Check that the array a is properly partitioned at position j
    n = len(a)
    assert ( j >= 0 and j < n) # otherwise this makes no sense to 
                               # say array a is partitioned at j
    fail = False
    x = a[j] # the value of the pivot
    if verbose:
        print('The pivot is:', x)
    # Check all elements before the pivot
    for k in range(0, j): # recall range in python runs from 0 to j-1
        if (a[k] > x):
            if (verbose):
                print( 'Partition fails at position: a[%d] =  %d'%(k, a[k]) )
            fail = True
    for k in range(j+1, n):
        if (a[k] < x):
            if (verbose):
                print('Partition fails at position a[%d] = %d'%(k, a[k]))
            fail = True
    if verbose:
        if (not fail):
            print('-> Partition is correct (trumpets please)')
        else:
            print('-> Partitioned failed (wawa trombones please)')
    return (not fail)

In [2]:
# Test 1
a = [1,4,6,7,1,15,4, 5, 7, 9]
j = 5
isPartitioned(a,j) # This is not partitioned since the pivot 15, has elements smaller than it to the right.

The pivot is: 15
Partition fails at position a[6] = 4
Partition fails at position a[7] = 5
Partition fails at position a[8] = 7
Partition fails at position a[9] = 9
-> Partitioned failed (wawa trombones please)


False

In [3]:
# Test 2
a = [ 1, 4, 6, 1, 3, 5, 6, 7, 8, 9, 8, 7, 10, 12, 11, 8]
j = 7 # This array is correctly partitioned around the pivot element 7 at a[7]
isPartitioned(a,j)

The pivot is: 7
-> Partition is correct (trumpets please)


True

## Partitioning Schemes

Each partitioning scheme starts from the given array, with designated pivot (the last element of the array) and proceeds to move elements around until the array is fully partitioned. Keep this picture in mind:

`[..Original Array..] ==> [.. Intermediate Array .. ] ==> [.. Final Partitioned Array..]`


## Lomuto Partitioning Scheme

This scheme is credited to _Nico Lomuto_  and is called the Lomuto scheme. It is simple, elegant and rather easy to code/get correct.

To understand this code, we need to comprehend the key property that it maintains:

- There are two indices or pointers `i` and `j` that will demarcate some regions of the array.
- Initially, `i = -1` and `j = 0`, but as the algorithm progresses, it will make sure that 
   `i <= n-1, i <= j, and j <= n-1`
- Let us call the region between `[0,i]` (both ends inclusive as denoted by the square brackets) region A and the region `[i+1, j-1]` region B.

$$ {\large[}\; \underset{\mbox{Region A}}{\underbrace{ a[0], a[1], \cdots, a[i] }},\ \ \underset{\mbox{Region B}}{\underbrace{a[i+1], \ldots, a[j-1]}},\ \underset{\mbox{To Be Processed}}{\underbrace{\mathbf{a[j]}, \ldots, a[n-2]}},\ \mathbf{x} \ {\large]} $$

The invariants are as follows:

- $\forall\ k \in [0,i]\; a[k] \leq x \,.$ _ Every element in region A is less than or equal to pivot _ 
- $\forall\ k \in [i+1,j-1]\; a[k] > x \,.$ _ Every element in region B is strictly greater than the pivot _ 
- $a[n-1] = x$, _ The pivot stays at position n-1 _ 

The algorithm itself proceeds by shrinking the ``to be processed region`` by one element each time. This is achieved at each step by taking `a[j]` the element at the head of this region and placing it in the correct place.

- **Case 1:** `a[j] <= x`: this element needs to go into region A. How do we do this? Here is what things look like at the start of the operation:
    <table style="border:none">
    <tr><td bgcolor="#FFCCCC"> <pre> <span style="bgcolor:red">a[0], a[1], .., a[i] </span> </pre> <td bgcolor="#CCCCFF"> <pre>  <span style="color:blue">a[i+1], a[i+2], ... , a[j-1]</span> </pre> <td> <pre> <span style="color:green"> a[j] </span>, ... rest </pre> </tr>
     <tr><td style="text-align:center"><pre> <span style="color:red"> |------- Region A ------| </span><td style="text-align: center"> <span style="color:blue"> |--------  Region B --------| </span> <td style="text-align:center"> |--- To be processed </pre>
     </table> 
     
* Swap `a[j]` with `a[i+1]`: 
    <table style="border:none">
    <tr><td bgcolor="#FFCCCC"> <pre> <span style="color:red">a[0], a[1], .., a[i] </span> <td bgcolor="#CCCCFF"> <pre> <span style="color:green">a[j]</span>, <span style="color:blue"> a[i+2], ... , a[j-1]</span></pre> <td> <pre> <span style="color:blue"> a[i+1] </span>, ... rest</pre>
     </table>
     
* Increment `i, j` moving regions A and B to the left.
    <table style="border:none">
    <tr><td bgcolor="#FFCCCC"> <pre> <span style="color:red">a[0], a[1], .., a[i], a[j] </span> </pre> <td bgcolor="#CCCCFF"><pre> <span style="color:blue"> a[i+2], ... , a[j-1]</span>, <span style="color:blue"> a[i+1] </span></pre><td> <pre> ... rest</pre>
     </table>






- **Case 2:** `a[j] > x`: this element needs to go into region B. Once again, here is how things look at the start of the operation:
  <table style="border:none">
    <tr><td> <pre> <span style="color:red">a[0], a[1], .., a[i] </span>, <span style="color:blue">a[i+1], a[i+2], ... , a[j-1]</span>, <span style="color:green"> a[j] </span>, ... rest</pre>
     <tr><td><pre> <span style="color:red">|---- Region A ------| </span><span style="color:blue">|--------  Region B ------| </span> |--- To be processed </pre>
     </table>
* Increment `j` to move the region B to the right and encompass this new element `a[j]`.
  <table style="border:none">
    <tr><td> <pre> <span style="color:red">a[0], a[1], .., a[i] </span>, <span style="color:blue">a[i+1], a[i+2], ... , a[j-1]</span>, <span style="color:blue"> a[j] </span>, ... rest</pre>
     <tr><td><pre> <span style="color:red">|---- Region A ------| </span><span style="color:blue">|--------  Region B --------------| </span> |--- To be processed </pre>
     </table>






In [146]:
from IPython.core.display import display, HTML

def printArrayHeader(n):
    rstr='<tr><td> <pre> Indices --> </pre> </td> '
    for k in range(0,n):
        rstr += '<td bgcolor=\"#AA99BB\"> %d </td>'%(k)
    rstr+= '</tr>'
    return rstr

def visualizeArray(a, i, j, last=False):
    n = len(a)
    rstr ='<tr><td> <pre> i=%d j=%d </pre> </td>'%(i,j)
    for k in range(0,n-1):
        if (k >= 0 and k <= i ):
            rstr += '<td bgcolor = \"red\">  %d  </span> </td>'%(a[k])
        elif (k == i+1 and last):
            rstr += '<td> <b> %d </b>  </td>'%(a[k])
        elif (k >= i+1 and k <= j-1):
            rstr += '<td bgcolor=\"blue\"> %d </span> </td>'%(a[k])
        else:
            rstr += ' <td> %d </td>'%(a[k])
    if (not last):
        rstr += '  <td> <b> %d </b> </td> '%(a[n-1])
    else:
        rstr += '<td bgcolor=\"blue\"> %d </span> </td>'%(a[n-1])
    #display(Markdown('\n-------------------\n'))
    return rstr
    

In [147]:
def swap(a, i, j):
    assert( i >= 0 and j >= 0 and i < len(a) and j < len(a))
    a[i], a[j] = a[j], a[i] # pythonic swap
    
def lomutoPartition(a, visualize=False):
    n = len(a)
    i = -1
    j = 0
    x = a[n-1]
    
    if visualize:
        rstr=''
        rstr += printArrayHeader(len(a))
        rstr += visualizeArray(a,i,j)
        
    while (j < n-1):
        if (a[j] <= x): # case-1
            swap(a, j, i+1)
            i = i + 1  # move regions A, B both to the left
            j = j + 1
        else: # case-2
            j = j + 1
        if visualize:
            rstr += visualizeArray(a,i,j)
    # we are not done yet, need to move the pivot, but where. 
    # Pivot rightfully belongs to the border between regions A and B.
    # so we swap the first element of region B with the pivot.
    swap(a, i+1, n-1)
    if visualize:
        rstr += visualizeArray(a,i,j,True)
        rstr += '</tbody></table>'
        display(HTML(rstr))
    
    return i+1 # return the position where the new pivot is

In [148]:
a = [12, 1, 13, 5, 6, 1, 3, 6, 7, 8, 12, 11, 14, 2, 8, 6 ]
print('Input:', a)
k = lomutoPartition(a,True)
print('Output: ',a)
isPartitioned(a,k)

Input: [12, 1, 13, 5, 6, 1, 3, 6, 7, 8, 12, 11, 14, 2, 8, 6]


Indices -->,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
i=-1 j=0,12,1,13,5,6,1,3,6,7,8,12,11,14,2,8,6
i=-1 j=1,12,1,13,5,6,1,3,6,7,8,12,11,14,2,8,6
i=0 j=2,1,12,13,5,6,1,3,6,7,8,12,11,14,2,8,6
i=0 j=3,1,12,13,5,6,1,3,6,7,8,12,11,14,2,8,6
i=1 j=4,1,5,13,12,6,1,3,6,7,8,12,11,14,2,8,6
i=2 j=5,1,5,6,12,13,1,3,6,7,8,12,11,14,2,8,6
i=3 j=6,1,5,6,1,13,12,3,6,7,8,12,11,14,2,8,6
i=4 j=7,1,5,6,1,3,12,13,6,7,8,12,11,14,2,8,6
i=5 j=8,1,5,6,1,3,6,13,12,7,8,12,11,14,2,8,6
i=5 j=9,1,5,6,1,3,6,13,12,7,8,12,11,14,2,8,6


Output:  [1, 5, 6, 1, 3, 6, 2, 6, 7, 8, 12, 11, 14, 13, 8, 12]
The pivot is: 6
-> Partition is correct (trumpets please)


True

In [149]:
a=[12,1,5,7,15,5]
print('Input:', a)
k = lomutoPartition(a,True)
print('Output: ',a)
isPartitioned(a,k)

Input: [12, 1, 5, 7, 15, 5]


Indices -->,0,1,2,3,4,5
i=-1 j=0,12,1,5,7,15,5
i=-1 j=1,12,1,5,7,15,5
i=0 j=2,1,12,5,7,15,5
i=1 j=3,1,5,12,7,15,5
i=1 j=4,1,5,12,7,15,5
i=1 j=5,1,5,12,7,15,5
i=1 j=5,1,5,5,7,15,12


Output:  [1, 5, 5, 7, 15, 12]
The pivot is: 5
-> Partition is correct (trumpets please)


True

In [184]:
import random

def randomlyTestLomutoScheme(size, nTrials):
    a = []
    for i in range(0,size):
        a.append(random.randint(0,size))
    # create a random array a
    for i in range(0, nTrials):
        random.shuffle(a)
        k = lomutoPartition(a)
        assert(isPartitioned(a,k,False))

size = 50
nTrials = 50000
print('Randomly testing on %d arrays of size %d ... '%(nTrials, size), end='')
randomlyTestLomutoScheme(size, nTrials)
print('Done')

Randomly testing on 50000 arrays of size 50 ... Done


## Enhancements

The main enhancement is to use a `for` loop instead of a `while` loop. Also, rather than partition the whole array from `0` to `n`, we partition from `left` to `right`.

In [158]:
def lomuto_partition(a, left, right):
    ''' Lomuto partition of array a from indices left to right-1. 
        Following Python convention, the left index is inclusive
        but right is not.'''
    n = len(a)
    assert( 0 <= left and left < right and right <= n )
    i = left - 1
    x = a[right -1]
    for j in range(left, right-1):
        if (a[j] <= x):
            swap(a, i + 1, j)
            i += 1
        # else nothing to do
    swap(a, i + 1, right -1 )
    return i + 1

In [155]:
a=[12,1,5,7,15,5]
print('Input:', a)
k = lomuto_partition(a, 0, len(a))
print('Output: ',a, k)
isPartitioned(a,k)

Input: [12, 1, 5, 7, 15, 5]
Output:  [1, 5, 5, 7, 15, 12] 2
The pivot is: 5
-> Partition is correct (trumpets please)


True

# Hoare Partitioning Scheme

In this partitioning scheme, we operate differently.  Once again, let `x` be the pivot element, chosen to be
`a[n-1]` for array a of length `n`. 
There are two regions, 
- __Region A__ that contains elements which are `<= x` and 
- __Region B__ that contains elements `>= x`.

We maintain two indices `i` and `j`.

- **Region A**: defined as all indices in the range `[0, i]` (both ends inclusive as shown by the square brackets).
  All array elements in this region are guaranteed to be ` <= x `
  
- **Region B**: defined as all indices in the range `[j, n-1]` (both ends inclusive as shown by the square brackets). All array elements in this region are guaranteed to be ` >= x`. Note that the pivot element itself belongs to this region.

The array then looks like this:

<table><tbody>
<tr><td> 0 <td> 1 <td> ... <td bgcolor="yellow"> <b> i </b>  <td> ... <td bgcolor="yellow"> <b> j </b> <td> ... <td> n-1 </tr>
<tr><td bgcolor="#CCCCFF"> a[0] <td bgcolor="#CCCCFF"> a[1] <td bgcolor="#CCCCFF"> ... <td bgcolor="#CCCCFF"> a[i] <td> ... <td bgcolor="#FFCCCC"> a[j] <td bgcolor="#FFCCCC"> ... <td bgcolor="red"> <b> x </b> </tr>
<tr><td colspan="4" bgcolor="#CCCCFF"> <pre> &lt;= x </pre> <td> <pre> to be processed </pre> <td bgcolor="#FFCCCC" colspan="3"> <pre> &gt;= x </pre> </td> 
</tbody></table>

## Simplified Version - 1

Initially, we will start by setting `i=-1` and `j = n-1`, reflecting that region A is empty to start off, whereas region B is not. Each iteration tries to shrink the middle `to be processed` elements as follows:

- If `a[i+1] < x` then `a[i+1]` can clearly belong to the region A. Therefore, `i` can be incremented.
- If `a[j-1] > x` then `a[j-1]` can clearly belong to the region B. Therefore, `j` can be decremented.
- If neither condition holds and `i+1 <= j-1` (i.e, the to be processed region is non-empty) then 
    * Clearly `a[i+1] >= x` and `a[j-1] <= x`. 
    * Thus, we can swap `a[i+1]` and `a[j-1]`.

The iteration continues while `i+1 <= j-1` or there is at least one element remaining in the to be processed region.
When it finishes, the pivot should be swapped with position `j`.


In [187]:
def hoare_partition_simple(a):
    n = len(a)
    i = -1
    j = n - 1
    x = a[j]
    while (i + 1 <=  j -1): # the to be processed area is not empty
        assert( 0 <= i + 1 and i +1 <= j -1 and j <= n-1 )
        if (a[i + 1] < x): 
            i = i + 1
        elif (a[j - 1] > x):
            j = j - 1
        else: # we already know that i+1 <= j-1, so now sweat in swapping them
            swap(a, i + 1, j - 1)
            i += 1
            j -= 1
            
    assert i <= j     # this always holds when we exit the main loop    
    swap(a, j, n - 1) # Put the pivot in the "right place"
    return j          # return that.

In [189]:
a=[4,3,12,1,5,7,15,5]
print('Input:', a)
k = hoare_partition_simple(a)
print('Output: ',a,k)

b =[3,3,3,3,3,3,3,3,3,3]
print('Input:', b)
k = hoare_partition_simple(b)
print('Output: ',b,k)

c = [4,5,6,7,8,9,10]
print('Input:',c)
k = hoare_partition_simple(c)
print('Output:', c,k)


Input: [4, 3, 12, 1, 5, 7, 15, 5]
Output:  [4, 3, 5, 1, 5, 7, 15, 12] 4
Input: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Output:  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 4
Input: [4, 5, 6, 7, 8, 9, 10]
Output: [4, 5, 6, 7, 8, 9, 10] 6


In [205]:

def randomlyTestHoareScheme(size, nTrials):
    a = []
    for i in range(0,size):
        a.append(random.randint(0,size/2))
    # create a random array a
    for i in range(0, nTrials):
        random.shuffle(a)
        k = hoare_partition_simple(a)
        assert(isPartitioned(a,k,False))

size = 40
nTrials = 10000
print('Randomly testing on %d arrays of size %d ... '%(nTrials, size), end='')
randomlyTestHoareScheme(size, nTrials)
print('Done')

Randomly testing on 10000 arrays of size 40 ... Done
